In [ ]:
import math
import random
import struct
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import numpy as np
import scipy.io.wavfile

SR = 48000

def show(data):
    plt.plot(range(len(data)), data)
    plt.show()

def audio(data): display(Audio(data, rate=SR))

def nplog10(x): return np.log10(x.clip(min=1e-30))

def get_fft_xy(data):
    x = np.fft.rfftfreq(len(data), d=1 / SR)
    y = 20 * nplog10(np.abs(np.fft.rfft(data)) / SR)
    return x, y
    
def fft(data):
    plt.plot(*get_fft_xy(data))
    plt.ylim(-120, 0)
    
def show2(data1, data2):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(12, 4, forward=True)
    ax1.plot(range(len(data1)), data1)
    ax2.plot(range(len(data2)), data2)
    plt.show()

def show21(data1, data2, data3):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(12, 4, forward=True)
    ax1.plot(range(len(data1)), data1)
    ax1.plot(range(len(data2)), data2)
    ax2.plot(range(len(data3)), data3)
    plt.show()
    
def show_fft2(data1, data2):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(12, 4, forward=True)
    ax1.plot(*get_fft_xy(data1))
    ax1.set_ylim([-120, 0])
    ax2.plot(*get_fft_xy(data2))
    ax2.set_ylim([-120, 0])
    plt.show()

# Простые приемы звукового синтеза и алгоритмической композиции на языке Питон

*Пётр Советов*, МИРЭА

Часть 1. **Синтез звука**

Часть 2. Алгоритмическая композиция

# Где сегодня используется процедурное аудио/алгоритмическая музыка?

* Создание звуковых эффектов, звукового фона в играх и в кино.
* Динамическая музыка в играх.
* Генеративная музыка: Брайан Ино, Autechre, Бьорк.
* Демосцена, low-complexity art.

<center><img src="img/scape.png" width="30%"></img></center>


# Из истории компьютерного звука

## Однобитный звук в реальном времени

* *1951*. Прямое управление динамиком в реальном времени через однобитный порт, Джефф Хилл (Geoff Hill), компьютер CSIRAC.
* *1962*. Имитация полифонического звучания с помощью чередования голосов, Питер Сэмсон (Peter Samson), компьютер PDP-1.
* *1987*. Пять и более голосов с синтезом тембров, Тим Фоллин (Tim Follin) и другие, компьютер ZX Spectrum.
<img src="img/beeper.png" width="10%"></img>

In [ ]:
display(Audio(filename="mp3/chronos.mp3"))

## Синтез звука в пакетном режиме

* *1957*. Порождение треугольной волны с записью на магнитную ленту и последующим вопроизведением через 12-битный ЦАП, программа MUSIC-1, Макс Мэтьюс (Max Mathews), компьютер IBM 704.
* *1967*. Изобретение FM-синтеза, Джон Чоунинг (John Chowning).
* *1968*. Язык звукового синтеза MUSIC V, книга The Technology of Computer Music (1969), Макс Мэтьюс (Max Mathews).
<center>
<img src="img/musicv.png" width="30%"></img>MUSIC V
</center>

## Цифровые синтезаторы

* *1976*. Аддитивный синтезатор Bell Labs Digital Synthesizer (Alles Synth, "синтезатор Аллеса"), Хэл Аллес (Hal Alles), Bell Labs, 32 голоса.
* *1977*. Синтезатор Samson Box, Питер Сэмсон (Peter Samson), аддитивный, FM- и другие виды синтеза, Stanford, 256 генераторов звука, 16 голосов.
* *1982*. Синтезатор Audio Signal Processor (ASP), Джеймс Мурер (James A. Moorer), LucasFilm.

<table>
<tr>
    <td width="50%"><center><img src="img/alles.png" width="50%"></img><b>Alles Synth</b></center>
    <td><center><img src="img/samson.png" width="30%"></img><b>Samson Box</b></center>
</table>



In [ ]:
display(Audio(filename="mp3/thx.mp3"))

## Звуковые чипы: прямоугольник вместо синусоиды

* *1977*. Atari TIA (Atari 2600,...), 2 голоса (**прямоугольная** форма волны).`
* *1979*. TI SN76489 (TI-99/4A, BBC Micro,...), 4 голоса (**прямоугольная** форма волны, шум).
* *1979*. Atari POKEY (Atari 8-bit,...), 4 голоса (**прямоугольная** форма волны, шум).
* *1979*. GI AY-38910 (ZX Spectrum, Yamaha MSX,...), 3 голоса (**прямоугольная** форма волны, шум).
* *1982*. MOS 6581 SID (Commodore C64), 3 голоса (**прямоугольная**/пилообразная/треугольная формы волны, шум, фильтр).
* *1984*. Ricoh 2A03 (Famicom/NES), 5 голосов (**прямоугольная**/треугольная формы волны, шум, проигрывание 7-битных сэмплов).

# Практика
Для синтеза звука будем использовать пакетный режим работы и тембры классических звуковых чипов.

In [ ]:
def sec(n): return int(n * SR) # Секунды в кол-во сэмплов, SR - частота дискретизации

class Square: # Генератор прямоугольной формы волны
    def __init__(self):
        self.phase = 0 # Фаза изменяется в пределах [0, 1)
       
    def next(self, freq, width=0.5): # next выдает очередной сэмпл
        # width задает ширину импульса (0.5 - центр периода)
        y = 2 * int(self.phase < width) - 1 # Значение в {-1, 1}
        self.phase = (self.phase + freq / SR) % 1
        return y

o1 = Square() # Сравнение звучания для разных значений ширины импульса
o2 = Square()
out1 = [o1.next(400, 0.5) for i in range(sec(1))]   
out2 = [o2.next(400, 0.1) for i in range(sec(1))]   
show2(out1[:sec(0.01)], out2[:sec(0.01)]); audio(out1)

In [ ]:
class Saw: # Простой генератор пилообразного сигнала
    def __init__(self):
        self.phase = 0
       
    def next(self, freq):
        y = 2 * self.phase - 1
        self.phase = (self.phase + freq / SR) % 1
        return y

o1 = Saw()
out = [o1.next(400) for i in range(sec(1))]
audio(out)
show(out[:sec(0.01)])

In [ ]:
class Env: # Генератор огибающей с параметрами: время атаки и время затухания
    def __init__(self, attack_time=sec(0.0001)): # attack_time - время нарастания звука
        self.time = 0
        self.attack_time = attack_time
        self.amp = 0

    def reset(self): # Нажата новая нота, сброс состояния генератора
        self.time = 0

    def next(self, decay_time): # decay_time - время затухания
        if self.time < self.attack_time:
            self.amp = min(self.amp + 1 / self.attack_time, 1)
        else:
            self.amp = max(self.amp - 1 / decay_time, 0)            
        self.time += 1
        return self.amp

In [ ]:
# Демонстрация работы огибающей
o1 = Square()
attack = 0.1
decay = 0.7
e1 = Env(sec(attack))
out1 = [e1.next(sec(decay)) for i in range(sec(1))]
e1.reset()
out2 = [e1.next(sec(decay)) * o1.next(20) for i in range(sec(1))]
e1.reset()
out3 = [e1.next(sec(decay)) * o1.next(200) for i in range(sec(1))]
show2(out1, out2)
audio(out3)

In [ ]:
# Явление наложения частот (aliasing)
o1 = Square()
out1 = [o1.next(480) for i in range(sec(1))]
out2 = [o1.next(485) for i in range(sec(1))]
show_fft2(out1, out2)
audio(out1)
audio(out2)

# В решении проблем с наложением частот поможет FM-синтез
<center><img src="img/fm.png" width="10%"></img></center>

In [ ]:
# FM-синтез
class Sin: # Генератор синусоидального сигнала
    def __init__(self):
        self.phase = 0
       
    def next(self, freq, pm=0): # pm задает фазовую модуляцию
        y = math.sin(self.phase + pm)
        self.phase = (self.phase + 2 * math.pi * freq / SR) % (2 * math.pi)
        return y

o1 = Sin()
o2 = Sin()
e1 = Env()
out = []

for i in range(sec(1)): # Синтез тембра колокола
    out.append(o2.next(200, 5 * o1.next(700) * e1.next(sec(1))))

audio(out)

In [ ]:
o1 = Saw() # Трюк с получением прямоугольного сигнала (понадобится далее)
o2 = Saw() # в виде разности 2 пилообразных сигналов
o2.phase = 0.5
out1 = [o1.next(2) for i in range(sec(1))]
out2 = [o2.next(2) for i in range(sec(1))]
out3 = [a - b for a, b in zip(out1, out2)]
show21(out1, out2, out3)

In [ ]:
class FMSaw: # Синтез пилообразного синала с помощью FM и обратной связи
    def __init__(self):
        self.op = Sin()
        self.fb = 0
       
    def next(self, freq, cutoff=1.8): # cutoff работает подобно частоте среза в фильтрах
        self.fb = self.op.next(freq, self.fb * cutoff)
        return self.fb

o1 = Saw() # Сравнение звучания простой "пилы" и ее FM-версии
o2 = FMSaw()
audio([o1.next(507) for i in range(sec(1))])
audio([o2.next(507) for i in range(sec(1))])

In [ ]:
class FMSquare: # "FM-прямоугольник",
    def __init__(self): # используется трюк с разностью 2 пилообразных сигналов
        self.op1 = Sin()
        self.op2 = Sin()
        self.op2.phase = math.pi
        self.fb1 = 0
        self.fb2 = 0
       
    def next(self, freq, cutoff=1.5, width=0):
        self.fb1 = self.op1.next(freq, self.fb1 * cutoff)
        self.fb2 = self.op2.next(freq, self.fb2 * cutoff + width)
        return self.fb1 - self.fb2
    
o1 = Square() # Сравнение звучания простого "прямоугольника" и его FM-версии 
o2 = FMSquare()
audio([o1.next(507) for i in range(sec(1))])
audio([o2.next(507) for i in range(sec(1))])

In [ ]:
o1 = Square() # Сравнение спектров простого "прямоугольника" и его FM-версии
o2 = FMSquare()
out1 = [o1.next(507) for i in range(sec(1))]
out2 = [o2.next(507) for i in range(sec(1))]
show_fft2(out1, out2)

In [ ]:
o1 = FMSquare() # Имитация SID-звучания
o2 = Sin() # Модуляция ширины импульса
out = [o1.next(500, width=o2.next(1.5) * 3) for i in range(sec(2))]
show(out)
audio(out)

In [ ]:
class Voice: # Голос с параметрами: осциллятор и огибающая
    def __init__(self, osc, env):
        self.osc = osc
        self.env = env
        self.reset(0, 0)

    def reset(self, freq, amp): # None обозначает продолжание звучания
        if freq is not None:
            self.env.reset()
            self.freq = freq
            self.amp = amp
    
    # on_time - время нажатой "клавиши", play_time - общее время звучания голоса
    def play(self, freq, on_time, play_time, amp=1):
        samples = []
        self.reset(freq, amp)
        for i in range(play_time):
            vol = self.amp * self.env.next(on_time)
            samples.append(vol * self.osc.next(self.freq))
        return samples

In [ ]:
def midi2freq(m): # Перевод MIDI-значения [0-127] в герцы
    return 440 * 2 ** ((m - 69) / 12)

class SID_voice: # Имитация звучания SID
    def __init__(self):
        self.osc = FMSquare()
        self.lfo = Sin()
        self.env = Env()
        self.reset(0, 0)
        
    reset = Voice.reset
        
    def play(self, freq, on_time, play_time, amp=1, pwm_freq=0.3, pwm_amp=3):
        lst = []
        self.reset(freq, amp)
        for i in range(play_time):
            pwm = self.lfo.next(pwm_freq) * pwm_amp
            vol = self.amp * self.env.next(on_time)
            lst.append(vol * self.osc.next(self.freq, width=pwm))
        return lst

v1 = SID_voice()
out = []

for i in range(13): # 12 нот звукоряда
    out += v1.play(midi2freq(60 + i), sec(0.2), sec(0.4))

audio(out)

In [ ]:
# Чтение простого текстового формата, в духе музыкальных трекеров
# Пример: "D-3 ... C-3 ... === ... E-3 ..."
TRACK_NAMES = dict(zip("c- c# d- d# e- f- f# g- g# a- a# b-".split(),
                   range(12)))

def name2freq(name, trans): # Имя ноты в духе "С-4" или "d#5"
    n, o = name[:2].lower(), int(name[2])
    return midi2freq(TRACK_NAMES[n] + 12 * (o + 1) + trans)

# Перевод текста в список значений частота или None (удержание ноты)
def parse_track(text, trans=0):
    return [name2freq(x, trans) if x[0].isalpha() else
            None for x in text.split()]

def load_track(filename, trans=0): # Загрузить и разобрать текст
    with open(filename) as f:
        return parse_track(f.read(), trans)

track1 = load_track("txt/bionic2.txt", -24)
v1 = Voice(FMSquare(), Env())
out = []

for f in track1: # Проигрывание изолированного голоса отрывка
    out += v1.play(f, sec(0.2), sec(0.05))

audio(out)

In [ ]:
track1 = load_track("txt/bionic1.txt", -24)
track2 = load_track("txt/bionic2.txt", -24)
v1 = Voice(FMSquare(), Env())
out = []
alt = 2 # Количество переключений между голосами

for i in range(len(track1)): # Прием с чередованием голосов
    for j in range(alt):
        out += v1.play(track1[i], sec(0.1), sec(0.03) // alt)
        out += v1.play(track2[i], sec(0.1), sec(0.03) // alt)

audio(out)

In [ ]:
# Микширование нескольких голосов
def mix(*tracks): return [sum(x) for x in zip(*tracks)]

track1 = load_track("txt/bionic1.txt", -24)
track2 = load_track("txt/bionic2.txt", -24)
v1 = SID_voice()
v2 = Voice(FMSquare(), Env())
out = []

for i in range(len(track1)): # Двухголосное исполнение отрывка
    p1 = v1.play(track1[i], sec(2), sec(0.06))
    p2 = v2.play(track2[i], sec(0.2), sec(0.06))
    out += mix(p1, p2)      

audio(out)

# Эффект эхо (delay)
<br>
<center>
<img src="img/delay.png" width="50%"></img>
</center>

In [ ]:
class Delay: # Эффект дилэй ("эхо")
    def __init__(self, size):
        self.line = [0] * size # Линия задержки размером size
        self.idx = 0

    # level - уровень эхо-сигнала, fb - уровень обратной связи
    def play(self, samples, level, fb=0.5):
        lst = []
        for x in samples:
            old = self.line[self.idx] # Самое старое значение из линии задержки
            lst.append(x + old * level) # Выдача результата
            self.line[self.idx] = old * fb + x # Обновленное значение в линии задержки
            self.idx = (self.idx + 1) % len(self.line)
        return lst

track = load_track("txt/galaga.txt")
o1 = Voice(FMSquare(), Env())
d1 = Delay(sec(0.16))
out = []

for f in track:
    out += d1.play(o1.play(f, sec(0.4), sec(0.08)), 0.6, 0.6)

audio(out)

In [ ]:
track1 = load_track("txt/robocop.txt", -12)
v1 = Voice(FMSquare(), Env())
d1 = Delay(sec(0.15))
out = []

for f in track1: # Еще один отрывок с эффектом дилей
    out += d1.play(v1.play(f, sec(0.6), sec(0.06)), 0.5)

audio(out)

# Генератор шума в Atari POKEY

<center>
<img src="img/lfsr.png" width="30%"></img><b>Регистр сдвига с линейной обратной связью (LFSR)</b>
</center>

In [ ]:
class LFSR: # Генератор шума на основе регистра сдвига с линейной обратной связью
    def __init__(self, bits, taps):
        self.bits = bits # Разрядность регистра
        self.taps = taps # Индексы для вычисления очередного бита
        self.state = 1 # Состояние регистра
        self.phase = 0
    
    def next(self, freq):
        y = self.state & 1
        self.phase += 2 * freq / SR
        if self.phase > 1:
            self.phase -= 1
            # Xor значений по индексам бит taps
            x = 0
            for b in (self.state >> i for i in self.taps):
                x ^= b
            self.state = (self.state >> 1) | ((~x & 1) << (self.bits - 1))
        return 2 * y - 1

In [ ]:
o1 = LFSR(bits=5, taps=[3, 4, 0]) # Демонстрация периодичности формы сигнала
out = [o1.next(500) for i in range(sec(1))]
show(out[:sec(0.1)])
audio(out)

In [ ]:
# Характерный для Atari 8-bit тембр
track = load_track("txt/mule.txt", 24)
v1 = Voice(LFSR(5, [3, 4, 0]), Env())       
out = []

for f in track:
    out += v1.play(f, sec(0.3), sec(0.05))

audio(out * 2)

In [ ]:
slide = 0.11

class Kick_voice: # Синтез бас-барабана
    def __init__(self):
        self.osc = FMSquare()
        self.env = Env()
        self.pitch_env = Env() # Частотная огибающая
        self.amp = 0
        
    def reset(self, freq, amp):
        if freq is not None:
            self.env.reset()
            self.pitch_env.reset()
            self.amp = amp
        
    def play(self, freq, play_time, amp=1):
        lst = []
        self.reset(freq, amp)
        for i in range(play_time):
            vol = self.amp * self.env.next(sec(slide))
            lst.append(vol * self.osc.next(180 * self.pitch_env.next(sec(slide))))
        return lst

v1 = Kick_voice()
out = []

for i in range(8):
    out += v1.play(0, sec(1/2))

audio(out)

In [ ]:
class Snare_voice: # Синтез малого барабана
    def __init__(self):
        self.bass = FMSquare() # Басовая составляющая тембра
        self.osc = LFSR(12, [7, 9, 1, 2, 3, 11]) # Высокочастотная составляющая
        self.env = Env()
        self.pitch_env = Env()
        self.amp = 0
        
    def reset(self, freq, amp):
        if freq is not None:
            self.env.reset()
            self.pitch_env.reset()
            self.amp = amp
        
    def play(self, freq, play_time, amp=1):
        lst = []
        self.reset(freq, amp)
        for i in range(play_time):
            vol = self.amp * self.env.next(sec(0.1))
            m = 0.5 * self.osc.next(16000 * self.pitch_env.next(sec(0.3)))
            m += self.bass.next(320 * self.pitch_env.next(sec(0.4)))
            lst.append(vol * m)
        return lst

In [ ]:
# Простая барабанная партия
v1 = Kick_voice()
v2 = Snare_voice()
out = []

for i in range(8):
    out += v1.play(0, sec(1/8)) + v1.play(0, sec(1/8)) + v2.play(0, sec(1/4))

audio(out)
scipy.io.wavfile.write("drums1.wav", SR, np.array(out))

# Конец первой части